In [1]:
import time

import bqplot as bq
import numpy as np
import pandas as pd

from IPython.display import display

In [2]:
def live_plot(title, fields, x="", y="", history=30):
    global drop_count
    drop_count=0
    
    x_sc = bq.LinearScale()
    y_sc = bq.LinearScale()

    ax_x = bq.Axis(label=x, scale=x_sc, grid_lines='solid')
    ax_y = bq.Axis(label=y, scale=y_sc, orientation='vertical', grid_lines='solid')

    df = pd.DataFrame(columns=['time']+fields)
        
    pts = bq.Lines(x=df.index, y=[df[f].values for f in fields], display_legend=True, scales={'x': x_sc, 'y': y_sc})
    fig = bq.Figure(axes=[ax_x, ax_y], marks=[pts], labels=fields, legend_location='top-left', title=title)
    
    def cb(data):
        global drop_count
        
        if len(df) > history:
            df.drop([drop_count], inplace=True)
            drop_count += 1
        
        df.loc[len(df)+drop_count] = [time.time()] + data
        
        with pts.hold_sync():
            pts.x = df.index 
            pts.y = [df[f] for f in fields]
    
    return fig, cb

In [3]:
import serial 
import json

device = serial.Serial("/dev/cu.usbmodem1411", baudrate=115200)

fields = ["CH1", "CH2", "CH3", "CH4"]
fig, cb = live_plot("SenseTemp Temperatures", fields, x="Time (sec)", y="Temperature (C)")

def poll():
    row = device.readline().decode('UTF-8')
    if row[0] == "[":
        cb(json.loads(row))

display(fig)

while True:
    poll()

Figure(axes=[Axis(label='Time (sec)', scale=LinearScale()), Axis(label='Temperature (C)', orientation='vertica…

KeyboardInterrupt: 